In [1]:
import array
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.callbacks import EarlyStopping
from keras.engine.topology import Input
from keras.engine.training import Model
from keras import losses, optimizers
from keras import backend as K
import math

Using TensorFlow backend.


In [2]:
from trainlib.generator import generate_training_data, generate_validation_data

Welcome to JupyROOT 6.10/09


In [3]:
config = tf.ConfigProto(intra_op_parallelism_threads=10, inter_op_parallelism_threads=10, \
                        allow_soft_placement=True, device_count = {'CPU': 10})
session = tf.Session(config = config)
K.set_session(session)

2018-03-20 10:19:53.966063: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-03-20 10:19:53.966100: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-03-20 10:19:53.966109: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.


In [4]:
def sample_cut(frame):
    return frame.loc[frame["nCleanedJetsPt30"] >= 2]

In [5]:
inpath = "/data_CMS/cms/wind/CJLST_NTuples/"
filename = "/ZZ4lAnalysis.root"
H1_files = ["VBFH125"]
H0_files = ["ggH125"]

H1_paths = [inpath + H1_file + filename for H1_file in H1_files]
H0_paths = [inpath + H0_file + filename for H0_file in H0_files]

In [6]:
branches = ["PFMET", "nCleanedJetsPt30"]

In [7]:
in_layer = Input(shape = (2,))
x = Dense(128, activation = "relu")(in_layer)
x = Dense(128, activation = "relu")(x)
x = Dense(128, activation = "relu")(x)
x = Dense(16, activation = "relu")(x)
out_layer = Dense(1, activation = "tanh", name = "out_layer")(x)
model = Model(in_layer, out_layer, name = "testmodel")

In [8]:
sgd = optimizers.SGD(lr = 0.1)
model.compile(loss = "mean_squared_error", optimizer = sgd, metrics = ["accuracy"])

In [9]:
train_gen = generate_training_data(H1_paths, H0_paths, branches, cut = sample_cut, chunks = 100)
val_gen = generate_validation_data(H1_paths, H0_paths, branches, cut = sample_cut, chunks = 100)

skimming /data_CMS/cms/wind/CJLST_NTuples/VBFH125/ZZ4lAnalysis.root
collection set up: 1 files, 62320 entries in total, 31160 of which will be used
skimming /data_CMS/cms/wind/CJLST_NTuples/ggH125/ZZ4lAnalysis.root
collection set up: 1 files, 110483 entries in total, 55241 of which will be used
skimming /data_CMS/cms/wind/CJLST_NTuples/VBFH125/ZZ4lAnalysis.root
collection set up: 1 files, 62320 entries in total, 31160 of which will be used
skimming /data_CMS/cms/wind/CJLST_NTuples/ggH125/ZZ4lAnalysis.root
collection set up: 1 files, 110483 entries in total, 55242 of which will be used


In [10]:
early_stop = EarlyStopping(monitor = 'val_loss',
                          patience = 10,
                          verbose = 1,
                          mode = 'auto')

In [11]:
ret = model.fit_generator(train_gen, steps_per_epoch = 128, epochs = 50, verbose = 2, validation_data = val_gen, validation_steps = 10, callbacks = [early_stop])

H1 contains 31160 entriesEpoch 1/50

H0 contains 55241 entries
using the following chunk sizes: (311 / 552)
H1 contains 31160 entries
H0 contains 55242 entries
using the following chunk sizes: (311 / 552)
45s - loss: 0.3040 - acc: 0.7013 - val_loss: 0.3019 - val_acc: 0.6980
Epoch 2/50


KeyboardInterrupt: 